# ports and adapters with command handler patterns

In [1]:
from typing import NamedTuple

Architecture style: ports and adapters
Design pattern: Command handler

keeping business logic in model objects, no bleeding into controllers, no fat manager classes. 

complexity upfront to avoid accidental complexity down the road

building an issue management system

> First story: as a user I want to be able to report a new issue

3 architechture principles

1. we will always define where our use case begins and ends. no business processes strewn all over the codebase
2. depend on abstractions, not concrete implementations
3. glue code is distinct from business logic. put it in the right place

define the domain model. encapsulates our shared understanding of the problem. uses agreed terminology. create a separate python package for our domain model, no dependencies on other layers.

outside domain model put srvices - stateless objects that do stuff to the domain.

finally adapter layer. code that drives the service layer. eg concrete implementation for talking to the db. connect our app to the world

`DB -> (Adapters ( Services ( Domain ) ) ) <- User`

a command handler is an object that orchestrates a business process. similar to a controller in MVC

create a **command object** - small object that represents a state-changing action that can happen in the system. no behavior, pure data structures. Commands are instructions from an external agent. they have imperative names. avoid 'create' 'update' 'delete' - these are technical terms. use the business languange.

In [2]:
class ReportIssueCommand(NamedTuple):
    reporter_name: str
    reporter_email: str
    problem_description: str

the command objects are part of the domain. they express the api of your domain. The only way to change state is through a command. they can ve created in many ways: POST, celery, etc.

Create a **command handler** for your command. a stateless object that orchestrate system behaviour. kind of like glue code. fetching and saving objects, notifying other areas of the system.

Each command has exactly one handler.

In [3]:
class ReportIsseCommandHandler:
    def __init__(self, issue_log):
        self.issue_log = issue_log
        
    def __call__(self, cmd):
        reported_by = IssueReporter(
            cmd.reporter_name,
            cmd.reporter_email
        )
        issue = Issue(reported_by, cmd.problem_description)
        self.issue_log.add(issue)

the structure of CHs is consistent:
1. fetch the current state from persistent storage
2. update current state
3. persist the new state
4. notify any external systems the state has changed.

Boring code, no ifs, loops etc. stick to a single line of execution. no business logic. The following would be a bad CH:

In [4]:
class MarkIssueAsResolvedHandler:
    def __init__(self, issue_log):
        self.issue_log = issue_log
        
    def __call__(self, cmd):
        issue = self.issue_log.get(cmd.issue_id)
        # following is business logic
        if (issue.state != IssueStatus.Resolved):
            issue.mark_as_resolved(cmd.resolution)

here the if statement belongs in the domain model, probably in `mark_as_resolved` method of our issue logic

1. commands are logic free structures - just a name and bunch of values
2. they are a simple stable API. they don't depend on implementation
3. commands are handled by exactly 1 handler
4. each command instructs run through 1 use case
5. a handler: fetches state, updates state, persists new state, notifies about change.

In [5]:
# domain model

class IssueReporter:
    def __init__(self, name, email):
        self.name = name
        self.email = email
        

class Issue:
    def __init__(self,reporter,description):
        self.description = description
        self.reporter = reporter
        

class IssueLog:
    def add(self, issue):
        pass


class ReportIssueCommand(NamedTuple):
    reporter_name: str
    reporter_email: str
    problem_description: str
        
# service layer

class ReportIsseCommandHandler:
    def __init__(self, issue_log):
        self.issue_log = issue_log
        
    def __call__(self, cmd):
        reported_by = IssueReporter(
            cmd.reporter_name,
            cmd.reporter_email
        )
        issue = Issue(reported_by, cmd.problem_description)
        self.issue_log.add(issue)
        
# adapter

class FakeIssueLog(IssueLog):
    def __init__(self):
        self.issues = []
        
    def add(self, issue):
        self.issues.append(issue)
        
    def get(self, id):
        return self.issues[id]
    
    def __len__(self):
        return len(self.issues)
    
    def __getitem__(self, idx):
        return self.issues[idx]

# repository and unit of work pattern

add persistent data access.

in our handler we had

```
reporter = IssueReporter(cmd.reporter_name, cmd.reporter_email)
issue = Issue(reporter, cmd.problem_description)
issue_log.add(issue)
```

the issue log comes from the business terminology. So it belongs in the domain. but its also the ideal abstraction for our data store. but we don't want our issue log to depend on our database. That leads us to ports and adapters.

in P&A, a domain exposes ports. a port gets data in or out of the domain. IssueLog was a port. Ports are connected to the world by adapters (FakeIssueLog)

a circuit which detects current over a threshold. if the threshold exceeds, output a signal. two ports, in and out.

In [6]:
class ReadablePort:
    pass

class WriteablePort:
    pass

class ThresholdDetectionCircuit:
    arbitrary_threshold = 4
    
    def __init__(self, input: ReadablePort, 
                 output: WriteablePort):
        self.input = input
        self.output = output
        
    def read_from_input(self):
        next_value = self.input.read()
        if next_value > self.arbitrary_threshold:
            self.output.write(1)

because your ports are standardised you can plug in different devices

In [7]:
class LightDetector(ReadablePort):
    def read(self):
        return self.get_light_amplitude()
    
class Buzzer(WriteablePort):
    def write(self, value):
        if value > 0:
            self.make-noise()
            
class Dial(ReadablePort):
    def read(self):
        return self.current_value
    
class Light(WriteablePort):
    def write(self, value):
        if value > 0:
            self.on = True
        else:
            self.on = False

Back to our project: IssueLog is like WriteablePort a way to get data in and out. below we plug in sqlalchemy and text. 

In [8]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker()

class SqlAlchemyIssueLog(IssueLog):
    def __init__(self, session: Session):
        self.session = session
        
    def add(self, issue):
        self.session.add(issue)
        
class TextFileIssueLog(IssueLog):
    def __init__(self, path):
        self.path = path
    
    def add(self, issue):
        with open(self.path, 'w') as f:
            json.dump(f)

In fact IssueLog is a **repository**, an object that hides the detail of persistent storage by giving us an interface that looks like a collection. you can add things to a repository, or get things out. thats it.

A simple repo pattern

In [10]:
class FooRepository:
    def __init__(self, db_session):
        self.db_session = db_session
        
    def add_new_item(self, item):
        self.db_session.add(item)
        
    def get_item(self, id):
        return self.db_session.get(Foo, id)
    
    def find_foos_by_latitude(self, latitude):
        return self.session.query(Foo).filter(foo.latitude == latitude)